# Does Specializing More than Once Improve Learning

In [1]:
from res_specialization import *
from matplotlib import pyplot as plt
import sys
import random
import pickle
plt.rcParams['figure.figsize'] = [10, 5]

In [2]:
TRIALS      = 1000
NUM_TO_SPEC = 3
TOL = 5

diff_eq_params = {"x0": [-20, 10, -.5], 
                  "begin": 0, 
                  "end": 60, 
                  "timesteps":60000}

res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True
             }

results = dict()


def save_results():
    pickle.dump(results, open("spec_tw.pkl","wb"))
# end

### Model trajectory

In [3]:
t, train_t, u = lorenz_equ(**diff_eq_params )
train_t = t[30000:55000]
test_t  = t[55000:]

### Re-run grant figure trials to get prediction length


In [4]:
random_err       = []
random_pred      = []
specialized_err  = []
specialized_pred = []


spec_sizes       = []
spec_edges       = []

for i in range(TRIALS):
    
    ######################
    # Random graph
    ######################
    
    # Make rc
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Make predicitions
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    
    # Store Error
    random_err.append(err)
    random_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))
    
    #######################
    # Specialize graph
    ######################
    
    A = specialize_best_nodes(rc, NUM_TO_SPEC, u, train_t, r_0=r_0)
    spec_sizes.append(A.shape[0])
    spec_edges.append(np.sum(A != 0))
    
    rc = make_res_comp(A, res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    specialized_err.append(err)
    specialized_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))


In [5]:
results["random_err"]       = random_err
results["random_pred"]      = random_pred
results["specialized_err"]  = specialized_err
results["specialized_pred"] = specialized_pred
results["spec_sizes"]       = spec_sizes
results["spec_edges"]       = spec_edges
save_results()
print("Mean Prediction Length: {}".format(np.mean(random_pred)))
print("Mean Prediction Length: {}".format(np.mean(specialized_pred)))

Mean Prediction Length: 175.961
Mean Prediction Length: 2463.711


### Control: Same sizes and same edge densities as previous experiment

In [6]:
match_edge_err = []
match_edge_pred = []

for s,e in zip(spec_sizes,spec_edges):
    # Test random graph with same number of edges as specialized graphs
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Predict
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    match_edge_err.append(err)
    match_edge_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))


In [7]:
results["match_edge_err"] = match_edge_err
results["match_edge_pred"] = match_edge_pred
print("Mean Prediction Length: {}".format(np.mean(match_edge_pred)))
save_results()


Mean Prediction Length: 1395.971


### Specialization without finding best nodes

In [8]:
res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True
             }

random_specialized_err  = []
random_specialized_pred = []
rspec_sizes         = []
rspec_edges         = []

for i in range(TRIALS):
    # Random graph
    rc = ResComp(3,3,**res_params)
    
    # Turn reservoir to integer adj matrix
    A = rc.res
    for j in range(A.shape[0]): A[j,j] = 0
    A = (A != 0)*1
    
    # Specialize the reservoir
    A = specializeGraph(A, random.sample(list(range(30)),27))
    
    # Store data
    rspec_sizes.append(A.shape[0])
    rspec_edges.append(np.sum(A != 0))
    
    # Make new reservoir
    rc = make_res_comp(A, res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Get predicitons
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    
    # Store data
    random_specialized_err.append(err)
    random_specialized_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))


In [9]:
results["random_specialized_err"]  = random_specialized_err
results["random_specialized_pred"] = random_specialized_pred
results["rspec_sizes"]             = rspec_sizes
results["rspec_edges"]             = rspec_edges

print("Mean Prediction Length: {}".format(np.mean(random_specialized_pred)))
save_results()


Mean Prediction Length: 2026.941


### Control: Same sizes and same edge densities as previous experiment


In [10]:
rand_match_edge_err  = []
rand_match_edge_pred = []

for s,e in zip(rspec_sizes, rspec_edges):
    # Test random graph
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Predict
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    rand_match_edge_err.append(err)
    rand_match_edge_pred.append(how_long_accurate(u(test_t), test_pre, tol=TOL))
    

In [11]:
results["rand_match_edge_err"]  = rand_match_edge_err
results["rand_match_edge_pred"] = rand_match_edge_pred
print("Mean Prediction Length: {}".format(np.mean(rand_match_edge_pred)))
save_results()


Mean Prediction Length: 1153.45


### Targeted Specialize Twice

In [ ]:
res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True
             }

spec_tw_pre = []
spec_tw_err = []
tw_edges = []
tw_sizes = []
HOW_MANY_SPEC = 2

for i in range(TRIALS):
    # Make rc
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Specialize 2 times
    for i in range(HOW_MANY_SPEC):
        A = specialize_best_nodes(rc, NUM_TO_SPEC, u, train_t, r_0=r_0)
        rc = make_res_comp(A, res_params)
        r_0 = rc.state_0
        err = rc.fit(train_t,u)
    
    # Predict the system states
    tw_sizes.append(A.shape[0])
    tw_edges.append(np.sum(A != 0))
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    spec_tw_err.append(err)
    spec_tw_pre.append(how_long_accurate(u(test_t), test_pre,tol=TOL))

In [ ]:
results["spec_tw_pre"] = spec_tw_pre
results["spec_tw_err"] = spec_tw_err
results["tw_edges"] = tw_edges
results["tw_sizes"] = tw_sizes
save_results()


In [ ]:
print("Mean Prediction Length: {}".format(np.mean(spec_tw_pre)))


### Control: Same sizes and same edge densities as previous experiment

In [ ]:
tw_match_edge_err  = []
tw_match_edge_pred = []

for s,e in zip(tw_sizes, tw_edges):
    # Test random graph
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Predict
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    tw_match_edge_err.append(err)
    tw_match_edge_pred.append(how_long_accurate(u(test_t),test_pre,tol=TOL))
    

In [ ]:
results["tw_match_edge_err"] = tw_match_edge_err
results["tw_match_edge_pred"] = tw_match_edge_pred
save_results()
print("Mean Prediction Length: {}".format(np.mean(tw_match_edge_pred)))


### Targeted Specialize Three Times

In [ ]:
res_params = {"res_sz": 30, 
              "activ_f": np.tanh,
              "connect_p": .12, 
              "ridge_alpha": .00001, 
              "spect_rad": .9, 
              "gamma": 1., 
              "sigma": 0.12,
              "uniform_weights": True
             }

spec_thr_pre = []
spec_thr_err = []
thr_edges = []
thr_sizes = []
HOW_MANY_SPEC = 3

for i in range(TRIALS):
    # Make RC
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    rc.fit(train_t,u)
    
    # Specialize 3 times
    for i in range(HOW_MANY_SPEC):
        A = specialize_best_nodes(rc, NUM_TO_SPEC, u, train_t, r_0=r_0)
        rc = make_res_comp(A, res_params)
        r_0 = rc.state_0
        err = rc.fit(train_t,u)
    
    # Make predicitons
    thr_sizes.append(A.shape[0])
    thr_edges.append(np.sum(A != 0))
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    spec_thr_err.append(err)
    spec_thr_pre.append(how_long_accurate(u(test_t), test_pre,tol=TOL))


In [ ]:
results["spec_thr_pre"] = spec_thr_pre
results["spec_thr_err"] = spec_thr_err
results["thr_edges"] = thr_edges
results["thr_sizes"] = thr_sizes
print("Mean Prediction Length: {}".format(np.mean(spec_thr_pre)))

save_results()


### Control: Same sizes and same edge densities as previous experiment

In [ ]:
thr_match_edge_err  = []
thr_match_edge_pred = []

for s,e in zip(thr_sizes, thr_edges):
    # Test random graph
    
    # Make and fit RC
    res_params["res_sz"] = s
    res_params["connect_p"] = e/(s**2)
    rc = ResComp(3,3,**res_params)
    r_0 = rc.state_0
    err = rc.fit(train_t,u)
    
    # Predict systems
    test_pre = rc.predict(test_t)
    train_pre = rc.predict(train_t, r_0=r_0)
    thr_match_edge_err.append(err)
    thr_match_edge_pred.append(how_long_accurate(u(test_t),test_pre,tol=TOL))
    

In [ ]:
results["thr_match_edge_err"] = thr_match_edge_err
results["thr_match_edge_pred"] = thr_match_edge_pred
print("Mean Prediction Length: {}".format(np.mean(thr_match_edge_pred)))

save_results()


## Organize results into dataframes


In [ ]:
import pandas as pd

In [ ]:
labels = ["random_specialized","specialized","spec_twice", "spec_three"]
df = pd.DataFrame(results)
df.describe()

### Error

In [ ]:
df[["random_specialized_err","specialized_err","spec_tw_err", "spec_thr_err"]].describe()

### Error Controls

In [ ]:
df[["rand_match_edge_err", "match_edge_err", "tw_match_edge_err", "thr_match_edge_err"]].describe()

### Prediction

In [ ]:
df[["random_specialized_pred","specialized_pred","spec_tw_pre", "spec_thr_pre"]].describe()

### Prediction Control

In [ ]:
df[["rand_match_edge_pred", "match_edge_pred", "tw_match_edge_pred", "thr_match_edge_pred"]].describe()

### Sizes

In [ ]:
sizes = df[["rspec_sizes", "spec_sizes", "tw_sizes", "thr_sizes"]]
sizes.describe()

### Edges

In [ ]:
edges = df[["rspec_edges", "spec_edges", "tw_edges", "thr_edges"]]
edges.describe()

### Connectivity: (num edges)/(num nodes)^2

In [ ]:
labels = ["random_specialized","specialized","spec_twice", "spec_three"]
edge_prob = pd.DataFrame(np.array(edges)/(np.array(sizes)**2),columns=labels)
edge_prob.describe()

In [ ]:
df[["random_err", "random_pred"]].describe()